$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [1]:
import numpy as np
import shutil
import pandas as pd
import glob
import os

personal_folder = os.getcwd()
paramcards_folder = os.path.join(os.path.dirname(personal_folder), 'Branching_Ratios_Plot/Paramcards_Betard_0_4')
MG5_Path = os.path.relpath('/Collider/MG5_aMC_v3_1_0/bin/mg5_aMC', os.getcwd())

In [2]:
M = ["1000", "1250", "1500", "1750", "2000", "2250", "2500"]
g_U = ["1.8"]

In [3]:
ruta_modelo = "/disco1/SIMULACIONES/LeptoQuarks/UFOs/Mod2_VLQ_UFO"
ruta_pythia = "/disco1/SIMULACIONES/LeptoQuarks/pythia8_card.dat"
ruta_delphes = "/disco1/SIMULACIONES/LeptoQuarks/delphes_card_CMS-0.7bTag.dat"

In [4]:
linea_zerowidth_tchannel = "set zerowidth_tchannel False"
linea_modelo = "import model " + ruta_modelo
linea_def1 = "define lq = vlq vlq~"
linea_def2 = "define ta = ta+ ta-"
linea_gen = "generate p p > ta ta @0 QED = 0 QCD = 0"
lineas_definiciones = [linea_zerowidth_tchannel, linea_modelo, linea_def1, linea_def2, linea_gen]

lineas_outputs = []
for i in range(len(M)):
    for j in range(len(g_U)):
        name="M" + M[i].replace('.','_') + "_gU" + g_U[j].replace('.','_')
        path=os.path.join(personal_folder,"Data_100K_13_TeV","ta_ta",name)
        output_ij = f"output {path} --no-jpeg"
        lineas_outputs.append(output_ij)
lineas = np.concatenate((lineas_definiciones, lineas_outputs))

script = open ('folders_data_ta_ta_M_gU.txt','w')
script.write(lineas[0])

for i in range(1,len(lineas)):
    script.write('\n' + lineas[i])
script.close()

In [5]:
!{MG5_Path} folders_data_ta_ta_M_gU.txt

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 3.1.0                 2021-03-30         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

In [6]:
!rm folders_data_ta_ta_M_gU.txt

In [7]:
!mkdir -p {personal_folder}/Data_100K_13_TeV/
!mkdir -p {personal_folder}/Data_100K_13_TeV/ta_ta/
!mkdir -p {personal_folder}/Data_100K_13_TeV/ta_ta/Cross_Sections/
!mkdir -p {personal_folder}/Data_100K_13_TeV/ta_ta/SubProcesses/
!mkdir -p {personal_folder}/Data_100K_13_TeV/ta_ta/HTML/

In [ ]:
seeds = []

def semilla():
    seed = np.random.randint(1,5000)
    if (seed in seeds):
        return semilla()
    return seed

for k in range(0,2):            
    for i in range(len(M)):
        for j in range(len(g_U)):
            
            etiqueta = "M" + M[i].replace('.','_') + "_gU" + g_U[j].replace('.','_')
            nombre_script = 'generate_data_tau_tau_' + etiqueta + '.txt'
            script = open (nombre_script,'w')

            launch = f"launch {personal_folder}/Data_100K_13_TeV/ta_ta/" + etiqueta + " -m"
            nodos = "6"
            shower = "shower = Pythia8"
            detector = "detector = Delphes"
            done = "done"

            seed = semilla()
            seeds.append(seed)
            param1 = "set iseed " + str(seed)
            param2 = "set nevents 1000"
            param3 = "set cut_decays True"
            param4 = "set ptb 30"
            param5 = "set ptj 30"
            param6 = "set ptl 30"
            param7 = "set etab 2.5"
            param8 = "set pt_min_pdg {15: 30}"
            param9 = "set eta_max_pdg {15: 2.5}"
            param10 = "set mmll 20"
            param11 = "set GZP 0"
            param12 = "set GGP 0"

            ruta_paramcard = f"{paramcards_folder}/" + "param_card_" + etiqueta + ".dat"

            lineas = [launch,nodos,shower,detector,done,param1, param2, param3, param4, param5, param6, param7, param8, param9, param10, param11, param12, ruta_paramcard, ruta_pythia, ruta_delphes]

            script.write(lineas[0])

            for k in range(1,len(lineas)):    
                script.write('\n' + lineas[k])

            script.close()

            !{MG5_Path} {nombre_script}
            !rm {nombre_script}
            
            source_folder = f"{personal_folder}/Data_100K_13_TeV/ta_ta/" + etiqueta + "/Events"
            target_folder = f"{personal_folder}/Data_100K_13_TeV/ta_ta/Events/" + etiqueta
            source_file_xs = f"{personal_folder}/Data_100K_13_TeV/ta_ta/" + etiqueta + "/crossx.html"
            target_file_xs = f"{personal_folder}/Data_100K_13_TeV/ta_ta/Cross_Sections/crossx_" + etiqueta + ".html"
            
            !rm -rf {target_file_xs}
            shutil.copy(source_file_xs, target_file_xs)
            !mkdir -p {target_folder}
            !rsync --modify-window=1 --update --recursive --copy-links {source_folder} {target_folder}                     

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 3.1.0                 2021-03-30         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

In [ ]:
for i in range(len(M)):
    for j in range(len(g_U)):
        etiqueta = "M" + M[i].replace('.','_') + "_gU" + g_U[j].replace('.','_')
        folder = f"{personal_folder}/Data_100K_13_TeV/ta_ta/" + etiqueta
        
        subprocesses_folder = folder + "/SubProcesses/"
        files = glob.iglob(os.path.join(subprocesses_folder, "**/*.ps"))

        for file_path in files:
            if os.path.isfile(file_path):
                ruta = os.path.dirname(os.path.abspath(file_path))
                for k in range(len(ruta)):
                    if (ruta[len(ruta) -1 - k] == "/"):
                        indice = len(ruta) -1 - k
                        subcarpeta = ruta[indice:]
                        break

                target_folder = f"{personal_folder}/Data_100K_13_TeV/ta_ta/SubProcesses/{etiqueta}{subcarpeta}"
                !mkdir -p {target_folder}
                shutil.copy(file_path, target_folder)

                source_file_config_map = ruta +  "/config_subproc_map.inc"
                target_file_config_map = target_folder +  "/config_subproc_map.inc"
                shutil.copy(source_file_config_map, target_file_config_map)
        
        HTML_folder = folder + "/HTML/"
        files = glob.iglob(os.path.join(HTML_folder, "**/results.html"))

        for file_path in files:
            if os.path.isfile(file_path):
                ruta = os.path.dirname(os.path.abspath(file_path))
                for k in range(len(ruta)):
                    if (ruta[len(ruta) -1 - k] == "/"):
                        indice = len(ruta) -1 - k
                        subcarpeta = ruta[indice:]
                        break

                target_folder = f"{personal_folder}/Data_100K_13_TeV/ta_ta/HTML/{etiqueta}{subcarpeta}"
                !mkdir -p {target_folder}
                shutil.copy(file_path, target_folder)            
        
        #shutil.rmtree(folder) #Borra la carpeta con los outputs de MadGraph

In [ ]:
xs = np.zeros((len(g_U), len(M)))
for j in range(len(M)):
    for i in range(len(g_U)):
        etiqueta = "M" + M[j].replace('.','_') + "_gU" + g_U[i].replace('.','_')  
        
        ruta_tabla = f"{personal_folder}/Data_100K_13_TeV/ta_ta/Cross_Sections/crossx_" + etiqueta + ".html"
        tabla_ij = pd.read_html(ruta_tabla)
        columna_xs = tabla_ij[0]['Cross section (pb)']
        xs[i,j] = float(columna_xs[0].split(' ')[0])         
        
XS_Matriz = pd.DataFrame(xs)
XS_Matriz.columns = M
XS_Matriz.index = g_U

XS_Matriz.to_excel(f"{personal_folder}/Data_100K_13_TeV/ta_ta/Cross_Sections/XS_Matriz.xlsx")
XS_Matriz.to_csv(f"{personal_folder}/Data_100K_13_TeV/ta_ta/Cross_Sections/XS_Matriz.csv")

In [ ]:
!touch termino_ta_ta.txt